In [1]:
import csv
import re


In [2]:
unique_nodes = []
nodes = []
with open("./linkComm_result/t3_class_central_res_node.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        nodes.append(dict({
            'id': row['node'],
            'cluster': 'c'+row['cluster']
        }))
        
        if row['node'] not in unique_nodes:
            unique_nodes.append(row['node'])
                     
        

In [3]:
final_nodes = []

for i, node in enumerate(unique_nodes):
    
    final_nodes.append(dict({
        'id': node,
        'community': '',
        'overlapping_num': 0,
        'communities': []
        
    }))
    
    # unique node list안의 node id로 탐색
    for n in nodes:
                
        if n['id'] == node:
            
            if final_nodes[i]['community'] == '':
                final_nodes[i]['community'] += n['cluster']
                final_nodes[i]['overlapping_num'] += 1
            else:
                final_nodes[i]['community'] += str("+" + n['cluster'])
                final_nodes[i]['overlapping_num'] += 1
                    
            
        
    

## assign sinlge community by neighbors proportion

In [4]:

edges = []
with open("./linkComm_result/t3_class_central_res_edge.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        edges.append(dict({
            'source': row['node1'],
            'target': row['node2'],
            'cluster': 'c'+row['cluster']
            
        }))
        
                     
        

In [5]:
for i, node in enumerate(final_nodes):
    
    for edge in edges:
        if node['id'] == edge['source'] or node['id'] == edge['target']:
            
            flag = False
            for j, c in enumerate(node['communities']):
                if c['cluster'] == edge['cluster']:
                    final_nodes[i]['communities'][j]['count'] += 1
                    flag = True
                
            if flag != True:
                final_nodes[i]['communities'].append(dict({
                    'cluster': edge['cluster'],
                    'count': 1
                }))
                
    # sort the communities list, and assign the single community by first element
    final_nodes[i]['communities'].sort(key=lambda i: i['count'], reverse=True)
    final_nodes[i]['single_community'] = final_nodes[i]['communities'][0]['cluster']

In [6]:
## remove the communities key
for i, node in enumerate(final_nodes):
    del final_nodes[i]['communities']


In [7]:

# write file as csv
with open('./linkComm_result/res_node_cc_t3.csv', 'w', newline = '') as write_file:
    
    fieldnames = ['id', 'community', 'single_community', 'overlapping_num']
    
    dict_writer = csv.DictWriter(write_file, fieldnames)
    dict_writer.writeheader()
    dict_writer.writerows(final_nodes)
